# <center> Decorators

A **decorator** is any <ins>callable object that is used to modify a function or class</ins>. A reference to a function or a class is passed to a decorator, and the decorator returns a modified function or class.  
The modified functions or classes usually contain calls to the original function or class.

We have <ins>two kinds</ins> of decorators in Python:   
. Function decorators   
. Class decorators   


Note: the material used in this noebook has been taken from: [link](https://www.python-course.eu/python3_decorators.php).

## 1. Functions in python
### 1.1 Functions inside functions

In [ ]:
# Function names are references to functions
def succ(x):
    return x + 1

# We can assign multiple names to the same functio
successor = succ
print('1) succ() and successor() give the same result:', successor(10), '=', succ(10))

# Removing succ() is not affecting successor() size they both refer to the same function indipendently
del succ
print('successor() is still available after deleting succ(): ', successor(10))

In [ ]:
# You can define functions inside of a function (not possible in C++)
def f():    
    def g():
        print("Hi, it's me 'g'")
        print("Thanks for calling me")
        
    print("This is the function 'f'")
    print("I am calling 'g' now:")
    g()

# Note the order of the printout.
f()

### 1.2 Functions as paramters

In [ ]:
#  It gets useful in combination with two further powerful possibilities of Python functions. Due to the fact that every parameter of a function is a reference to an object and functions are objects as well, we can pass functions - or better "references to functions" - as parameters to a function.
import math

def foo(func):
    print("The function " + func.__name__ + " was passed to foo")
    res = 0
    for x in [1, 2, 2.5]:
        res += func(x)
    return res

print(foo(math.sin))
print(foo(math.cos))

### 1.3 Functions returning Functions

In [ ]:
# The output of a function is also a reference to an object: Functions can return references to function objects.
def polynomial_creator(*coeffs):
    # Coefficients are in the form a_n, a_n_1, ... a_1, a_0 
    def polynomial(x):
        res = coeffs[0]
        for i in range(1, len(coeffs)):
            res = res * x + coeffs[i]
        return res
                 
    return polynomial

p1 = polynomial_creator(4)  # p1 is a function. You call p1(value) and you get a number.
p2 = polynomial_creator(2, 4)  # Same for p2
p3 = polynomial_creator(1, 8, -1, 3, 2)  # Same for p3

x = 1.5
print(f'x={x}; p1(x)={p1(x)}; p2(x)={p2(x)}; p3(x)={p3(x)};')

## 2 Introduction to decorators

In [ ]:
# Decorators are functions that takes other function as input. They return a modified version of the input function.
def our_decorator(func):  # The decorator takes func as input
    def function_wrapper(x):  # It contains another fucntion 
        print("Before calling " + func.__name__)
        func(x) 
        print("After calling " + func.__name__)
    return function_wrapper  # The decorator returns a new function, that calls the original func and does somethign extra

def foo(x):
    print("Hi, foo has been called with " + str(x))

print("1) We call foo('Hi') before decoration:")
foo("Hi")
    
print("\n2) We now decorate foo")
foo = our_decorator(foo)  # foo is now function_wrapper(x).

print("\n3) We call foo(42) after decoration:")
foo(42)

**NOTE**: The decoration in Python is usually not performed in the way we did it in our previous example, since "foo" existed in the same program in two versions, before decoration and after decoration.  
Instead of writing the statement: ```  foo = our_decorator(foo) ```
we can write:   ```  @our_decorator    ```. Note that his line has to be directly positioned in front of the decorated function.

In [ ]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling " + func.__name__)
        func(x)
        print("After calling " + func.__name__)
    return function_wrapper

@our_decorator
def foo(x):
    print("Hi, foo has been called with " + str(x))

foo("Hi")  # Sicne you decorated foo, you are actually calling function_wrapper(x)

## 3 Use cases for decorators
### 3.1 Checking Arguments with a Decorator

In [ ]:
def argument_test_natural_number(f):
    def helper(x):
        if type(x) == int and x > 0:
            return f(x)
        else:
            print("Argument is not an integer")
    return helper
    
@argument_test_natural_number
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

print('factorial(5)', factorial(5))
print('factorial(-1)', factorial(-1))

### 3.2 Counting Function Calls with Decorators

In [ ]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1  # Every time you use the decorated function calls is increased by one
        return func(*args, **kwargs)
    helper.calls = 0  # This get called only when the decorator is placed

    return helper

@call_counter
def succ(x):
    return x + 1

@call_counter
def mul1(x, y=1):
    return x*y + 1

print('succ.calls =', succ.calls)
for i in range(10):
    succ(i)
mul1(3, 4)
mul1(4)
mul1(y=3, x=2)
    
print('succ.calls =', succ.calls)
print('mul1.calls =', mul1.calls)

### 3.3 Decorators with Parameters
We want to generalize the decorator, by giving him a parameter.  
It menas we have to wrap another function around our previous decorator function to accomplish this.


In [ ]:
def greeting(expr):  # The parameter will be defined once you apply the decorator with: @greeting("καλημερα")
    def greeting_decorator(func):
        def function_wrapper(x):
            print(expr + ", " + func.__name__ + " returns:")
            func(x)
        return function_wrapper
    return greeting_decorator  # You return: greeting_decorator(function_wrapper(x))

@greeting("καλημερα")
def foo(x):
    print(42)

foo("Hi") # You are executing greeting_decorator(function_wrapper('Hi'))

### 3.4 Using wraps from functools
The way we have defined decorators so far hasn't taken into account that the attributes:   
. **\_\_name\_\_**: name of the function  
. **\_\_doc\_\_**: the docstring  
. **\_\_module\_\_**: The module in which the function is defined  
of the original functions will be lost after the decoration.


In [ ]:
# Example where the attributes are lost, and the decorators one are used instead
def greeting(func):
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper

@greeting
def f(x):
    """ just some silly function """
    return x + 4

print(f(10))
print("function name: " + f.__name__)  # It reflects function_wrapper
print("docstring: " + f.__doc__)
print("module name: " + f.__module__) 

**SOLUTION**: We can save the original attributes of the function f, if we assign them inside of the decorator. 
We change our previous decorator accordingly and save it as

In [ ]:
def greeting(func):
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    function_wrapper.__name__ = func.__name__  # If you want these attributes to change you need to impose it manually
    function_wrapper.__doc__ = func.__doc__
    function_wrapper.__module__ = func.__module__
    return function_wrapper 

@greeting
def f1(x):
    """ just some silly function """
    return x + 4

print(f1(10))
print("function name: " + f1.__name__)  # It reflects function_wrapper
print("docstring: " + f1.__doc__)
print("module name: " + f1.__module__) 

**NOTE**: this seems quite a bit of work, so you can actually using a default python method to make this change. We can import the decorator "wraps" from functools instead and decorate our function in the decorator with it:

In [ ]:
from functools import wraps

def greeting(func):
    @wraps(func)  # This is transferring all the func methods value to function_wrapper for us.
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper

@greeting
def f1(x):
    """ just some silly function """
    return x + 4

print(f1(10))
print("function name: " + f1.__name__)  # It reflects function_wrapper
print("docstring: " + f1.__doc__)
print("module name: " + f1.__module__) 

## 4. Classes instead of Functions
Not only functions are callable objects. A callable object is an object which can be used and behaves like a function but might not be a function.  
It is possible to define classes in a way that the instances will be callable objects. This is done using the ```__call__``` method.




In [ ]:
class A:
    def __init__(self):
        print("An instance of A was initialized")
    
    def __call__(self, *args, **kwargs):
        print("Arguments are:", args, kwargs)
              
x = A()
print("Now calling the instance:")
x(3, 4, x=11, y=10)

### 4.1 Using a Class as a decorator

In [ ]:
# This decorator is a function
def decorator1(f):
    def helper():
        print("Decorating", f.__name__)
        f()
    return helper

@decorator1
def foo():
    print("inside foo()")

foo()

In [ ]:
# This decorator is a Class
class decorator2:
    def __init__(self, f):  # The function to modify is passed in the constructor
        self.f = f
    def __call__(self):
        print("Decorating", self.f.__name__)  # You czall the original function with the __call_ method
        self.f()

@decorator2
def foo():
    print("inside foo()")

foo() 